<a href="https://colab.research.google.com/github/fvirtuozo/ML_for_Project_Time_Prediction/blob/main/Scripts_for_portifolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

import pandas as pd
import re
import os
import tabulate
import time
import numpy as np
import random
import string
from faker import Faker

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, r2_score


from google.colab import drive

from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default
import gspread

Need to Prepare in this project:


1.   Merge Datasets Asist Origin and Asist Open (data from ticket manager separates in 2 files)
2.   Convert time in seconds to Days
3. Extract the Project Number from the Jira Summary (worklogs and projects) - this will link the table Jira with the table Asist
4. Add the categorization of who is Developer and who is Tester, and from which project each belongs
5. Convert DateTime formats to  format='%d/%m/%Y'
6. Select Columns to be Used



Goal of the analysis:

1. Create a pivot table that show the Classification of projects/TMA depending on their original estimate time (+ than 5 days is a TMA)
2. Create a table that shows per person and per week how many days were logged in Jira
3. Create a table with the non facturable days
4. Show per project for ECATS: Time Spent on Development, Original estimate of project, Time of development spent on Bugs opened by Staci, time of development spent on bugs opened by HN, Time spent on Testes vs original estimate, Number of bugs opened by HN and Staci, AVG solution time
5. Apply ML to predict how many time to solve new tickets and a classification model by Priority or urgent

#Importing Data Sources and Creating data Output

In [ ]:
drive.mount('/content/drive')
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Mounted at /content/drive


In [ ]:
tabela_asist = '1pJEjDJ1q4C5hyMfGTEYanFT_4BtmK0QAWyCFY3pc2ps'
tabela_jira = '1QqnqGQO5SHtnIvB_X-CoGIZ4s5aECBSFkHzdXIQYZeA'
tabela_worklogs = '12QVQ0PgcU2KEDkXxm9UX-RLwL8nT-HHzl8IMmwqAev8'

In [ ]:
asist_output = gc.open_by_key(tabela_asist).sheet1
jira_output = gc.open_by_key(tabela_jira).sheet1
worklogs_output = gc.open_by_key(tabela_worklogs).sheet1

In [ ]:
worklogs = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Portifolio/Cópia de worklogs_2025.xlsx')

worklogs_edited = gc.open_by_key(tabela_worklogs).sheet1
worklogs_edited = pd.DataFrame(worklogs_edited.get_all_records())

db_asist_origin = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Portifolio/Cópia de Asist.xlsx', header = None)
db_asist_open = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Portifolio/Cópia de asist_open.xlsx', header = None)

df_Projects = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Portifolio/Cópia de Jira Export Excel CSV.csv')
folder_to_save = r'.\Data_Sources'

#Exploration of data - Project's Database

In [ ]:
df_Projects

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project lead id,...,Comment.36,Comment.37,Comment.38,Comment.39,Comment.40,Comment.41,Parent,Parent summary,Status Category,Status Category Changed
0,Testing/Recette,RCT75-211,65818,Story,To Do,RCT75,STACI : RECATS2375,software,Andre Rodrigues Diogo,712020:c488220b-1277-4c83-8d54-cf5459debf12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,To Do,11/Mar/25 3:32 PM
1,"Forecast creation and edition, Article and Tra...",RCT76-152,65813,Bug,To Do,RCT76,STACI : RECATS2376,software,Andre Rodrigues Diogo,712020:c488220b-1277-4c83-8d54-cf5459debf12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,To Do,11/Mar/25 1:28 PM
2,Reference and designation lost after editing t...,RCT76-151,65809,Bug,To Do,RCT76,STACI : RECATS2376,software,Andre Rodrigues Diogo,712020:c488220b-1277-4c83-8d54-cf5459debf12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,To Do,11/Mar/25 12:37 PM
3,Forecast article missing product reference in ...,RCT76-150,65807,Bug,In Progress,RCT76,STACI : RECATS2376,software,Andre Rodrigues Diogo,712020:c488220b-1277-4c83-8d54-cf5459debf12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Progress,11/Mar/25 11:49 AM
4,TEST - 2503100034,SS-263,65799,Tâche,A faire,SS,STACI : Socle Augusta,software,Joel Alvaredo,5f69367acf5999006aacd3ad,...,NaN,NaN,NaN,NaN,NaN,NaN,65796.0,2503100034 - Correction BL spécifiques ALMS po...,To Do,10/Mar/25 4:53 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Testing/Recette,RCT75-8,60680,Sous-tâche,RECETTE (TESTING) STACI,RCT75,STACI : RECATS2375,software,Andre Rodrigues Diogo,712020:c488220b-1277-4c83-8d54-cf5459debf12,...,NaN,NaN,NaN,NaN,NaN,NaN,60677.0,2403060031 - [ECATS] Bug Quota Budget,In Progress,13/May/24 10:26 AM
996,Backend - Refactor the validateOffer,RCT75-7,60679,Sous-tâche,RECETTE (TESTING) STACI,RCT75,STACI : RECATS2375,software,Andre Rodrigues Diogo,712020:c488220b-1277-4c83-8d54-cf5459debf12,...,NaN,NaN,NaN,NaN,NaN,NaN,60677.0,2403060031 - [ECATS] Bug Quota Budget,In Progress,17/Apr/24 6:39 PM
997,Backend - Refactor web/modules/custom/,RCT75-6,60678,Sous-tâche,RECETTE (TESTING) STACI,RCT75,STACI : RECATS2375,software,Andre Rodrigues Diogo,712020:c488220b-1277-4c83-8d54-cf5459debf12,...,NaN,NaN,NaN,NaN,NaN,NaN,60677.0,2403060031 - [ECATS] Bug Quota Budget,In Progress,16/Apr/24 6:46 PM
998,2403060031 - [ECATS] Bug Quota Budget,RCT75-5,60677,Story,RECETTE (TESTING) HN OK,RCT75,STACI : RECATS2375,software,Andre Rodrigues Diogo,712020:c488220b-1277-4c83-8d54-cf5459debf12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In Progress,16/Apr/24 1:06 PM


In [ ]:
colunas = [
    'Summary',
    'Issue key',
    'Issue id',
    'Issue Type',
    'Status',
    'Project key',
    'Parent',
    'Project name',
    'Assignee',
    'Creator',
    'Created',
    'Updated',
    'Σ Original Estimate',
    'Σ Time Spent',
]

In [ ]:
#Selecting Columns
df_Projects = df_Projects[colunas]
df_Projects

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Parent,Project name,Assignee,Creator,Created,Updated,Σ Original Estimate,Σ Time Spent
0,Testing/Recette,RCT75-211,65818,Story,To Do,RCT75,NaN,STACI : RECATS2375,Andre Rodrigues Diogo,Joao Gomes,11/Mar/25 3:32 PM,11/Mar/25 3:34 PM,NaN,86400.0
1,"Forecast creation and edition, Article and Tra...",RCT76-152,65813,Bug,To Do,RCT76,NaN,STACI : RECATS2376,Andre Rodrigues Diogo,Bruno Pires Amado,11/Mar/25 1:28 PM,12/Mar/25 11:23 AM,NaN,NaN
2,Reference and designation lost after editing t...,RCT76-151,65809,Bug,To Do,RCT76,NaN,STACI : RECATS2376,Andre Rodrigues Diogo,Bruno Pires Amado,11/Mar/25 12:37 PM,11/Mar/25 12:57 PM,NaN,NaN
3,Forecast article missing product reference in ...,RCT76-150,65807,Bug,In Progress,RCT76,NaN,STACI : RECATS2376,Andre Rodrigues Diogo,Bruno Pires Amado,11/Mar/25 11:45 AM,11/Mar/25 12:55 PM,NaN,NaN
4,TEST - 2503100034,SS-263,65799,Tâche,A faire,SS,65796.0,STACI : Socle Augusta,NaN,José Rodrigues,10/Mar/25 4:53 PM,11/Mar/25 3:09 PM,17280.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Testing/Recette,RCT75-8,60680,Sous-tâche,RECETTE (TESTING) STACI,RCT75,60677.0,STACI : RECATS2375,Ana Lourenço,Sophie Vieira,16/Apr/24 12:55 PM,14/Feb/25 11:28 AM,90000.0,113400.0
996,Backend - Refactor the validateOffer,RCT75-7,60679,Sous-tâche,RECETTE (TESTING) STACI,RCT75,60677.0,STACI : RECATS2375,Manuel Fernandes,Sophie Vieira,16/Apr/24 12:55 PM,14/Feb/25 11:28 AM,57600.0,59400.0
997,Backend - Refactor web/modules/custom/,RCT75-6,60678,Sous-tâche,RECETTE (TESTING) STACI,RCT75,60677.0,STACI : RECATS2375,Manuel Fernandes,Sophie Vieira,16/Apr/24 12:53 PM,14/Feb/25 11:28 AM,57600.0,72000.0
998,2403060031 - [ECATS] Bug Quota Budget,RCT75-5,60677,Story,RECETTE (TESTING) HN OK,RCT75,NaN,STACI : RECATS2375,Manuel Fernandes,Sophie Vieira,16/Apr/24 12:51 PM,08/Nov/24 4:35 PM,518400.0,453600.0


In [ ]:
df_Projects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Summary              1000 non-null   object 
 1   Issue key            1000 non-null   object 
 2   Issue id             1000 non-null   int64  
 3   Issue Type           1000 non-null   object 
 4   Status               1000 non-null   object 
 5   Project key          1000 non-null   object 
 6   Parent               366 non-null    float64
 7   Project name         1000 non-null   object 
 8   Assignee             932 non-null    object 
 9   Creator              1000 non-null   object 
 10  Created              1000 non-null   object 
 11  Updated              1000 non-null   object 
 12  Σ Original Estimate  534 non-null    float64
 13  Σ Time Spent         813 non-null    float64
dtypes: float64(3), int64(1), object(10)
memory usage: 109.5+ KB


In [ ]:
#Changing data types
df_Projects['Created'] = pd.to_datetime(df_Projects['Created'])
df_Projects['Updated'] = pd.to_datetime(df_Projects['Updated'])
df_Projects['Issue id'] = df_Projects['Issue id'].astype(object)
df_Projects['Parent'] = df_Projects['Parent'].astype(object)

<ipython-input-10-bb6ce599e002>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_Projects['Created'] = pd.to_datetime(df_Projects['Created'])
<ipython-input-10-bb6ce599e002>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Projects['Created'] = pd.to_datetime(df_Projects['Created'])
<ipython-input-10-bb6ce599e002>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_Projects['Updated'] = pd.to_datetime(df_Projects['Updated'])
<ipython-input-10-bb6ce599e002>:3: Settin

In [ ]:
#Replacing NA Values
df_Projects['Σ Original Estimate'] = df_Projects['Σ Original Estimate'].fillna(0)
df_Projects['Σ Time Spent'] = df_Projects['Σ Time Spent'].fillna(0)
df_Projects['Parent'] = df_Projects['Parent'].fillna('N/A')
df_Projects[ 'Assignee'] = df_Projects['Assignee'].fillna('N/A')

<ipython-input-11-7f69c1593c39>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Projects['Σ Original Estimate'] = df_Projects['Σ Original Estimate'].fillna(0)
<ipython-input-11-7f69c1593c39>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Projects['Σ Time Spent'] = df_Projects['Σ Time Spent'].fillna(0)
<ipython-input-11-7f69c1593c39>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [ ]:
df_Projects

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Parent,Project name,Assignee,Creator,Created,Updated,Σ Original Estimate,Σ Time Spent
0,Testing/Recette,RCT75-211,65818,Story,To Do,RCT75,N/A,STACI : RECATS2375,Andre Rodrigues Diogo,Joao Gomes,2025-03-11 15:32:00,2025-03-11 15:34:00,0.0,86400.0
1,"Forecast creation and edition, Article and Tra...",RCT76-152,65813,Bug,To Do,RCT76,N/A,STACI : RECATS2376,Andre Rodrigues Diogo,Bruno Pires Amado,2025-03-11 13:28:00,2025-03-12 11:23:00,0.0,0.0
2,Reference and designation lost after editing t...,RCT76-151,65809,Bug,To Do,RCT76,N/A,STACI : RECATS2376,Andre Rodrigues Diogo,Bruno Pires Amado,2025-03-11 12:37:00,2025-03-11 12:57:00,0.0,0.0
3,Forecast article missing product reference in ...,RCT76-150,65807,Bug,In Progress,RCT76,N/A,STACI : RECATS2376,Andre Rodrigues Diogo,Bruno Pires Amado,2025-03-11 11:45:00,2025-03-11 12:55:00,0.0,0.0
4,TEST - 2503100034,SS-263,65799,Tâche,A faire,SS,65796.0,STACI : Socle Augusta,N/A,José Rodrigues,2025-03-10 16:53:00,2025-03-11 15:09:00,17280.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Testing/Recette,RCT75-8,60680,Sous-tâche,RECETTE (TESTING) STACI,RCT75,60677.0,STACI : RECATS2375,Ana Lourenço,Sophie Vieira,2024-04-16 12:55:00,2025-02-14 11:28:00,90000.0,113400.0
996,Backend - Refactor the validateOffer,RCT75-7,60679,Sous-tâche,RECETTE (TESTING) STACI,RCT75,60677.0,STACI : RECATS2375,Manuel Fernandes,Sophie Vieira,2024-04-16 12:55:00,2025-02-14 11:28:00,57600.0,59400.0
997,Backend - Refactor web/modules/custom/,RCT75-6,60678,Sous-tâche,RECETTE (TESTING) STACI,RCT75,60677.0,STACI : RECATS2375,Manuel Fernandes,Sophie Vieira,2024-04-16 12:53:00,2025-02-14 11:28:00,57600.0,72000.0
998,2403060031 - [ECATS] Bug Quota Budget,RCT75-5,60677,Story,RECETTE (TESTING) HN OK,RCT75,N/A,STACI : RECATS2375,Manuel Fernandes,Sophie Vieira,2024-04-16 12:51:00,2024-11-08 16:35:00,518400.0,453600.0


In [ ]:
df_Projects['Σ Original Estimate'] = (df_Projects['Σ Original Estimate']/28800).round(2)
df_Projects['Σ Time Spent'] = (df_Projects['Σ Time Spent']/28800).round(2)


<ipython-input-13-02002db48ed8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Projects['Σ Original Estimate'] = (df_Projects['Σ Original Estimate']/28800).round(2)
<ipython-input-13-02002db48ed8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Projects['Σ Time Spent'] = (df_Projects['Σ Time Spent']/28800).round(2)


In [ ]:
#Extracting the Asist Project Number

def ticket_number(column):
    # Encontra todas as sequências de 10 números no texto
    numeros = re.findall(r'\d{10}', column)

    # Retorna a primeira sequência encontrada (se houver)
    return numeros[0] if numeros else None

In [ ]:
df_Projects['Nº Ticket Asist'] = df_Projects['Summary'].apply(ticket_number)
df_Projects['Nº Ticket Asist'] = df_Projects['Nº Ticket Asist'].astype(str)

<ipython-input-15-8e6db48c2493>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Projects['Nº Ticket Asist'] = df_Projects['Summary'].apply(ticket_number)
<ipython-input-15-8e6db48c2493>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Projects['Nº Ticket Asist'] = df_Projects['Nº Ticket Asist'].astype(str)


#Exploration and wrangling of Worklogs Database

In [ ]:
worklogs

,Worklog Id,Issue Key,Issue Summary,Issue Status,Time Spent,Time Spent (seconds),Start Date,Update Date,Comment,Author,...,Original Estimate,Original Estimate (seconds),Remaining Estimate,Remaining Estimate (seconds),Due Date,Parent Key,Parent Summary,Story Points,Fix Versions,Custom fields are available in Clockwork Pro
0,51500,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),00:30,1800,2025-01-27T11:07:00+01:00,2025-01-27T11:37:05+01:00,NaN,Diogo Afonso,...,1d 5h 36m,48960,0m,0,2025-01-23,NaN,NaN,NaN,NaN,NaN
1,51514,SA-393,2501240026 - [API] intégrateur API client: sta...,Livré (MEP),01:00,3600,2025-01-27T17:07:30+01:00,2025-01-27T18:07:35+01:00,NaN,Sérgio Matos,...,3h 12m,11520,1h 42m,6120,NaN,NaN,NaN,NaN,NaN,NaN
2,51534,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),08:00,28800,2025-01-28T05:24:15+01:00,2025-01-28T13:24:31+01:00,NaN,João Silvestre,...,1d 5h 36m,48960,0m,0,2025-01-23,NaN,NaN,NaN,NaN,NaN
3,51535,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),03:00,10800,2025-01-28T10:24:35+01:00,2025-01-28T13:24:39+01:00,NaN,João Silvestre,...,1d 5h 36m,48960,0m,0,2025-01-23,NaN,NaN,NaN,NaN,NaN
4,51561,SA-342,"2409270009 - Extract-Resubmit easily the ""payl...",QUALIFICATION,08:00,28800,2025-01-29T10:26:25+01:00,2025-01-29T18:26:32+01:00,NaN,João Silvestre,...,3w 2d 6h,511200,3w 7h,457200,NaN,SA-342,"2409270009 - Extract-Resubmit easily the ""payl...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,53169,RAUG2501-5,2402190004 - [RAUG] Correctif stockage mode ra...,RECETTE HN OK,02:00,7200,2025-03-11T14:11:07+01:00,2025-03-11T16:11:12+01:00,NaN,Diogo Afonso,...,NaN,0,0m,0,NaN,NaN,NaN,NaN,NaN,NaN
1330,53186,RAUG2501-26,TEST - 2503030040,To Do,03:00,10800,2025-03-11T15:48:39+01:00,2025-03-11T19:00:09+01:00,NaN,Diogo Afonso,...,3h 12m,11520,12m,720,NaN,NaN,NaN,NaN,NaN,NaN
1331,53185,RAUG2501-32,Chiffrage - 2503030051,To Do,00:35,2100,2025-03-11T18:18:28+01:00,2025-03-11T18:53:32+01:00,NaN,José Rodrigues,...,NaN,0,0m,0,NaN,NaN,NaN,NaN,NaN,NaN
1332,53179,RAUG2501-7,TEST - 2402190004,To Do,00:15,900,2025-03-11T18:21:33+01:00,2025-03-11T18:36:46+01:00,NaN,Beatriz Tomaz,...,1d,28800,3h 45m,13500,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
worklogs.columns

Index(['Worklog Id', 'Issue Key', 'Issue Summary', 'Issue Status',
       'Time Spent', 'Time Spent (seconds)', 'Start Date', 'Update Date',
       'Comment', 'Author', 'Author's Account Id', 'Project Key',
       'Project Name', 'Components', 'Labels', 'Epic Key', 'Epic Summary',
       'Issue Type', 'Original Estimate', 'Original Estimate (seconds)',
       'Remaining Estimate', 'Remaining Estimate (seconds)', 'Due Date',
       'Parent Key', 'Parent Summary', 'Story Points', 'Fix Versions',
       'Custom fields are available in Clockwork Pro'],
      dtype='object')

In [ ]:
columns = ['Worklog Id',
           'Issue Key',
           'Issue Summary',
           'Issue Status',
           'Time Spent (seconds)',
           'Start Date',
           'Update Date',
           'Author',
           'Project Name',
            'Issue Type',
           'Due Date',
            'Parent Key']

In [ ]:
worklogs = worklogs[columns]

In [ ]:
worklogs['Time Spent (Jours)'] = (worklogs['Time Spent (seconds)']/28800).round(2)
worklogs = worklogs.drop(columns=['Time Spent (seconds)'])

<ipython-input-20-c44b766be348>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  worklogs['Time Spent (Jours)'] = (worklogs['Time Spent (seconds)']/28800).round(2)


In [ ]:
worklogs['Start Date'] = pd.to_datetime(worklogs['Start Date']).dt.tz_localize(None)
worklogs['Week Number'] = worklogs['Start Date'].dt.isocalendar().week
worklogs['Year'] = worklogs['Start Date'].dt.isocalendar().year

In [ ]:
project_devs = {
    'Alex Mesquita': 'ECATS',
    'Andre Rodrigues Diogo': 'ECATS',
    'Beatriz Tomaz': 'Augusta',
    'Carlos Freixo': 'ECATS',
    'Daniel Marin': 'ECATS',
    'Diogo Afonso': 'Augusta',
    'Diogo Albino': 'API',
    'Joao Gomes': 'ECATS',
    'João Monteiro': 'ECATS',
    'João Silvestre': 'API',
    'Joel Alvaredo': 'Augusta',
    'José Rodrigues': 'Augusta',
    'Paulo Nakata': 'ECATS',
    'Pedro Barbosa': 'ECATS',
    'rkessal': 'ECATS',
    'Sérgio Matos': 'Augusta',
    'Soraia Esteves': 'Augusta',
    'Ana Domingues': 'Todos',
    'Ana Sousa Neto': 'ECATS',
    'Bruno Pires Amado': 'ECATS',
    'Gilberto Fernandes': 'API',
    'Chevalier': 'Todos'
}

In [ ]:
worklogs['Project'] = worklogs['Author'].map(project_devs)

In [ ]:
#Extracting Asist Ticket Number
worklogs['Nº Ticket Asist'] = worklogs['Issue Summary'].apply(ticket_number)
worklogs['Nº Ticket Asist'] = worklogs['Nº Ticket Asist'].astype(str)

In [ ]:
worklogs.isna().sum()

,0
Worklog Id,0
Issue Key,0
Issue Summary,0
Issue Status,0
Start Date,0
Update Date,0
Author,0
Project Name,0
Issue Type,0
Due Date,1175


In [ ]:
worklogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1334 entries, 0 to 1333
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Worklog Id          1334 non-null   int64         
 1   Issue Key           1334 non-null   object        
 2   Issue Summary       1334 non-null   object        
 3   Issue Status        1334 non-null   object        
 4   Start Date          1334 non-null   datetime64[ns]
 5   Update Date         1334 non-null   object        
 6   Author              1334 non-null   object        
 7   Project Name        1334 non-null   object        
 8   Issue Type          1334 non-null   object        
 9   Due Date            159 non-null    object        
 10  Parent Key          244 non-null    object        
 11  Time Spent (Jours)  1334 non-null   float64       
 12  Week Number         1334 non-null   UInt32        
 13  Year                1334 non-null   UInt32      

In [ ]:
worklogs.columns = worklogs.columns.str.strip()

In [ ]:
worklogs['Parent Key'] = worklogs['Parent Key'].fillna('N/A')
 #Due Date can have n/a values because will not be used in

In [ ]:
print(worklogs.columns)

Index(['Worklog Id', 'Issue Key', 'Issue Summary', 'Issue Status',
       'Start Date', 'Update Date', 'Author', 'Project Name', 'Issue Type',
       'Due Date', 'Parent Key', 'Time Spent (Jours)', 'Week Number', 'Year',
       'Project', 'Nº Ticket Asist'],
      dtype='object')


In [ ]:
worklogs

,Worklog Id,Issue Key,Issue Summary,Issue Status,Start Date,Update Date,Author,Project Name,Issue Type,Due Date,Parent Key,Time Spent (Jours),Week Number,Year,Project,Nº Ticket Asist
0,51500,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),2025-01-27 11:07:00,2025-01-27T11:37:05+01:00,Diogo Afonso,STACI : TMA_DEX ( API ),Epic,2025-01-23,N/A,0.06,5,2025,Augusta,2501150024
1,51514,SA-393,2501240026 - [API] intégrateur API client: sta...,Livré (MEP),2025-01-27 17:07:30,2025-01-27T18:07:35+01:00,Sérgio Matos,STACI : TMA_DEX ( API ),Story,NaN,N/A,0.12,5,2025,Augusta,2501240026
2,51534,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),2025-01-28 05:24:15,2025-01-28T13:24:31+01:00,João Silvestre,STACI : TMA_DEX ( API ),Epic,2025-01-23,N/A,1.00,5,2025,API,2501150024
3,51535,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),2025-01-28 10:24:35,2025-01-28T13:24:39+01:00,João Silvestre,STACI : TMA_DEX ( API ),Epic,2025-01-23,N/A,0.38,5,2025,API,2501150024
4,51561,SA-342,"2409270009 - Extract-Resubmit easily the ""payl...",QUALIFICATION,2025-01-29 10:26:25,2025-01-29T18:26:32+01:00,João Silvestre,STACI : TMA_DEX ( API ),Epic,NaN,SA-342,1.00,5,2025,API,2409270009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,53169,RAUG2501-5,2402190004 - [RAUG] Correctif stockage mode ra...,RECETTE HN OK,2025-03-11 14:11:07,2025-03-11T16:11:12+01:00,Diogo Afonso,STACI : RAUG2501,Epic,NaN,N/A,0.25,11,2025,Augusta,2402190004
1330,53186,RAUG2501-26,TEST - 2503030040,To Do,2025-03-11 15:48:39,2025-03-11T19:00:09+01:00,Diogo Afonso,STACI : RAUG2501,Tâche,NaN,N/A,0.38,11,2025,Augusta,2503030040
1331,53185,RAUG2501-32,Chiffrage - 2503030051,To Do,2025-03-11 18:18:28,2025-03-11T18:53:32+01:00,José Rodrigues,STACI : RAUG2501,Tâche,NaN,N/A,0.07,11,2025,Augusta,2503030051
1332,53179,RAUG2501-7,TEST - 2402190004,To Do,2025-03-11 18:21:33,2025-03-11T18:36:46+01:00,Beatriz Tomaz,STACI : RAUG2501,Tâche,NaN,N/A,0.03,11,2025,Augusta,2402190004


#Edit Asist Databases

### Closed Tickets

In [ ]:
db_asist_origin

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,Liste des tickets de 01/01/2024 à 12/03/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,État des tickets: Cloturé,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Date de l'extraction: 12/03/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Service destinataire,Type,Numéro,Date ouverture,Date prise en charge,Date résolution,Date cloture,Service demandeur,Code Entrepôt,Commettant,...,Sous-catégorie,Domaine,Titre,Désignation,Temps passé TMA,Temps passé Staci,Temps total centième,Temps total,Groupe Affectation,Motif
4,DSG,Demande,2503100029,10/03/2025 15:14,11/03/2025 09:53,12/03/2025 09:09,12/03/2025 09:09,UTILISATEUR,STACI V,9011 - CONSOMMABLES STACI SALON,...,Telephonie portable,AUTRES,demande de tél portable pour le site de MIRAMA...,NaN,00:00:00,00:00:00,0,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10950,DSI,Demande,2102040024,04/02/2021 13:57,21/06/2024 13:58,21/06/2024 13:58,26/06/2024 00:00,DTR,PUBLI 5,8050001 - BOUYGUES TELECOM,...,NaN,DT,CODE BARRE etiquette LM2S - BT,"Bjr, Lm2s nous remonte l'anomalie suivante sur...",00:00:00,00:00:00,0,00:00:00,"DSI, DSI PROJET",NaN
10951,DSI,Projet,2101220021,22/01/2021 15:43,13/12/2024 13:53,13/12/2024 13:53,18/12/2024 00:00,"DSI, DSI DAA, DSI TMA URGENCE, ADMIN",DSI,DIRECTION DES SYSTEMES D'INFORMATION -,...,NaN,AUTRES,Projet global API/BUS : Lot 3 = mise en place ...,Synthèse globale du lot 3 :Définir l’architect...,00:00:00,00:00:00,0,00:00:00,DSI PROJET,NaN
10952,DSI,Incident,2007060022,06/07/2020 13:20,09/07/2020 17:38,27/03/2024 16:28,30/03/2024 00:00,UTILISATEUR,STACI DE,DE4006 - ORIENTIS,...,NaN,AUGUSTA,"Problem with ""German Umlaute"" in Augusta","Bonjour,we have a Problem with the ""german Uml...",00:00:00,00:30:00,50,00:30:00,DSI,NaN
10953,DSI,Projet,2003180020,18/03/2020 16:37,18/03/2020 16:41,07/08/2024 11:27,09/08/2024 12:00,"DSI, DSI PROJET, DSI TMA URGENCE, ADMIN",DSI,NaN,...,NaN,AUGUSTA,[RAUG] PRO_MARCHANDS : PICKING SUR STOCK AVANCE,"bonjour, Ci joint le besoin pour le traitement...",00:00:00,00:00:00,0,00:00:00,"DSI PROJET, TMA",NaN


In [ ]:
#Removing First Lines
db_asist_origin.rename(columns = db_asist_origin.iloc[3], inplace=True)
db_asist_origin = db_asist_origin.drop([0, 1, 2,3], axis=0)
db_asist_origin.reset_index(drop=True, inplace=True)

In [ ]:
db_asist = db_asist_origin

In [ ]:
db_asist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10951 entries, 0 to 10950
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Service destinataire  10951 non-null  object
 1   Type                  10951 non-null  object
 2   Numéro                10951 non-null  object
 3   Date ouverture        10951 non-null  object
 4   Date prise en charge  10951 non-null  object
 5   Date résolution       10951 non-null  object
 6   Date cloture          10951 non-null  object
 7   Service demandeur     10951 non-null  object
 8   Code Entrepôt         10951 non-null  object
 9   Commettant            8042 non-null   object
 10  Compte                1104 non-null   object
 11  Catégorie             10944 non-null  object
 12  Sous-catégorie        2207 non-null   object
 13  Domaine               10849 non-null  object
 14  Titre                 10951 non-null  object
 15  Désignation           10834 non-null

In [ ]:
#Adding a column with the Status , due to the fact that this table has all the closed tickets
db_asist.loc[:,'Statut'] = 'Résolu'
db_asist.loc[:,'Priorité'] = np.random.choice(['Normale', 'Haute', 'Bas'], size=10951)

In [ ]:
db_asist.loc[:,'Date ouverture'] = pd.to_datetime(db_asist['Date ouverture'].str[:10], format='%d/%m/%Y')
db_asist.loc[:,'Date prise en charge'] = pd.to_datetime(db_asist['Date prise en charge'].str[:10], format='%d/%m/%Y')
db_asist.loc[:,'Date résolution'] = pd.to_datetime(db_asist['Date résolution'].str[:10], format='%d/%m/%Y')
db_asist.loc[:,'Date cloture'] = pd.to_datetime(db_asist['Date cloture'].str[:10], format='%d/%m/%Y')

In [ ]:
def time_to_jour(time_str):
    # Split the string by ":" to get hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_str.split(":"))

    # Convert to total seconds
    total_jour = (hours + minutes /60 + seconds/3600)/8
    return total_jour

time_str = "00:00:00"
db_asist.loc[:,'Temps passé TMA'] = db_asist['Temps passé TMA'].apply(time_to_jour)
db_asist.loc[:,'Temps passé Staci'] = db_asist['Temps passé Staci'].apply(time_to_jour)
db_asist.loc[:,'Temps total'] = db_asist['Temps total'].apply(time_to_jour)


In [ ]:
db_asist

,Service destinataire,Type,Numéro,Date ouverture,Date prise en charge,Date résolution,Date cloture,Service demandeur,Code Entrepôt,Commettant,...,Titre,Désignation,Temps passé TMA,Temps passé Staci,Temps total centième,Temps total,Groupe Affectation,Motif,Statut,Priorité
0,DSG,Demande,2503100029,2025-03-10 00:00:00,2025-03-11 00:00:00,2025-03-12 00:00:00,2025-03-12 00:00:00,UTILISATEUR,STACI V,9011 - CONSOMMABLES STACI SALON,...,demande de tél portable pour le site de MIRAMA...,NaN,0.0,0.0,0,0.0,NaN,NaN,Résolu,Haute
1,DSI,Demande,2503100006,2025-03-10 00:00:00,2025-03-10 00:00:00,2025-03-10 00:00:00,2025-03-12 00:00:00,UTILISATEUR,PUBLI 1,8016600 - JACOBS DOUWE EGBERTS FRANCE SAS,...,[R13]CHANGEMENT ADRESSE MAIL SUR LA BOUTIQUE,"Bonjour,Le client à acces à un formulaire a re...",0.0,0.020833,17,0.020833,DSI RUN,NaN,Résolu,Bas
2,DSI,Demande,2503100003,2025-03-10 00:00:00,2025-03-10 00:00:00,2025-03-10 00:00:00,2025-03-12 00:00:00,UTILISATEUR,STACI 3,3795 - BAXTER SAS,...,3487 - DEREFERENCEMENT,"Bonjour,pourriez-vous prendre en charge le CMA...",0.020833,0.0,17,0.020833,TMA MOP,NaN,Résolu,Bas
3,DSI,Demande,2503070045,2025-03-07 00:00:00,2025-03-10 00:00:00,2025-03-10 00:00:00,2025-03-12 00:00:00,UTILISATEUR,PUBLI 1,60029 - UNI-MEDIAS,...,[R13] MODIFCATION ADRESSE MAIL DEMANDE SUPPRES...,"Bonjour,Pouvez-vous modifier l'adresse mail de...",0.0,0.020833,17,0.020833,DSI RUN,NaN,Résolu,Normale
4,DSI,Demande,2503070043,2025-03-07 00:00:00,2025-03-10 00:00:00,2025-03-10 00:00:00,2025-03-12 00:00:00,UTILISATEUR,PUBLI 2,11108 - FREE,...,Extrac cde avec BTEACCESNEW 07/03,Jean Marc &nbsp;Peux-tu me sortir au plus vite...,0.0,0.0625,50,0.0625,DSI RUN,NaN,Résolu,Normale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10946,DSI,Demande,2102040024,2021-02-04 00:00:00,2024-06-21 00:00:00,2024-06-21 00:00:00,2024-06-26 00:00:00,DTR,PUBLI 5,8050001 - BOUYGUES TELECOM,...,CODE BARRE etiquette LM2S - BT,"Bjr, Lm2s nous remonte l'anomalie suivante sur...",0.0,0.0,0,0.0,"DSI, DSI PROJET",NaN,Résolu,Haute
10947,DSI,Projet,2101220021,2021-01-22 00:00:00,2024-12-13 00:00:00,2024-12-13 00:00:00,2024-12-18 00:00:00,"DSI, DSI DAA, DSI TMA URGENCE, ADMIN",DSI,DIRECTION DES SYSTEMES D'INFORMATION -,...,Projet global API/BUS : Lot 3 = mise en place ...,Synthèse globale du lot 3 :Définir l’architect...,0.0,0.0,0,0.0,DSI PROJET,NaN,Résolu,Normale
10948,DSI,Incident,2007060022,2020-07-06 00:00:00,2020-07-09 00:00:00,2024-03-27 00:00:00,2024-03-30 00:00:00,UTILISATEUR,STACI DE,DE4006 - ORIENTIS,...,"Problem with ""German Umlaute"" in Augusta","Bonjour,we have a Problem with the ""german Uml...",0.0,0.0625,50,0.0625,DSI,NaN,Résolu,Bas
10949,DSI,Projet,2003180020,2020-03-18 00:00:00,2020-03-18 00:00:00,2024-08-07 00:00:00,2024-08-09 00:00:00,"DSI, DSI PROJET, DSI TMA URGENCE, ADMIN",DSI,NaN,...,[RAUG] PRO_MARCHANDS : PICKING SUR STOCK AVANCE,"bonjour, Ci joint le besoin pour le traitement...",0.0,0.0,0,0.0,"DSI PROJET, TMA",NaN,Résolu,Normale


### Asist Open Tickets

In [ ]:
db_asist_open.rename(columns = db_asist_open.iloc[0], inplace=True)
db_asist_open = db_asist_open.drop([0], axis=0)
db_asist_open.reset_index(drop=True, inplace=True)

In [ ]:
db_asist_open.rename(columns={'Date/heure prise en charge (vide si non renseignée)': 'Date prise en charge',
                     'Date/heure résolution (vide si non renseignée)': 'Date résolution', 'Date/heure cloture (vide si non renseignée)':'Date cloture',
                             'Numéro ticket':'Numéro', "Groupe(s) d'attribution":'Groupe Affectation'}, inplace = True)

In [ ]:
db_asist_open

,Code entrepôt,Commettant,Compte,Type,Service destinataire,Catégorie,Sous-catégorie,Priorité,Numéro,Titre,...,Domaine,Environnement,Ticket(s) lié(s),Date ouverture,Date prise en charge,Date résolution,Date cloture,Date/heure de la dernière modification,Auteur de la dernière modification,Date d'intervention
0,DSI,NaN,NaN,Projet,DSI,DEX/Data EXchange,NaN,Normale,2503070046,BORDEREAU MEP DEROGATOIRE 11/03/2025,...,Autres,Production,"2502100020, 2502190039, 2502190040, 2502210031...",07/03/2025 17:38,NaN,NaN,NaN,11/03/2025 10:04,GAUTHIER Alexandre,NaN
1,STACI IT,NaN,NaN,Projet,DSI,DEX/Data EXchange,NaN,Normale,2503120018,[RDEX][1][1SPECEDI] EDI DPV : SUIVI DPV ORDINI...,...,EDI_SPEC,Production,2412310011,12/03/2025 13:24,12/03/2025 13:32,NaN,NaN,12/03/2025 13:32,ALVAREDO Joel,NaN
2,DSI,DIRECTION DES SYSTEMES D'INFORMATION -,NaN,Projet,DSI,DEX/Data EXchange,NaN,Normale,2503110035,[DEX] SEPIA - SAG flux stockStatus,...,API_SPEC,Production,NaN,11/03/2025 17:51,11/03/2025 18:02,NaN,NaN,12/03/2025 14:18,Silvestre Joao,NaN
3,DSI,DIRECTION DES SYSTEMES D'INFORMATION -,NaN,Projet,DSI,DEX/Data EXchange,NaN,Normale,2503110013,Check adapter charset to database,...,API_STD,Production,NaN,11/03/2025 11:30,11/03/2025 13:01,NaN,NaN,11/03/2025 13:01,FERNANDES Gilberto,NaN
4,DSI,NaN,NaN,Projet,DSI,Socle,NaN,Normale,2503100034,[RAUG2501]_Correctif BL spécifiques ALMS pour ...,...,Augusta,Production,2503060023,10/03/2025 16:37,10/03/2025 16:43,NaN,NaN,11/03/2025 15:12,RODRIGUES Jose,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,DSI,DIRECTION DES SYSTEMES D'INFORMATION -,NaN,Projet,DSI,Socle,NaN,Normale,2109080024,[DWH] - Monstro - Nouvelles Tables : Gestion ...,...,Autres,Production,2106070034,08/09/2021 15:45,08/09/2021 15:45,NaN,NaN,10/12/2024 16:47,MESQUITA Pedro,NaN
136,DSI,DIRECTION DES SYSTEMES D'INFORMATION -,NaN,Projet,DSI,Socle,NaN,Normale,2108060021,[RAUG] Correctif sur les mécanisme Purge des d...,...,Autres,Production,2107270002,06/08/2021 17:28,06/08/2021 17:31,NaN,NaN,20/02/2025 10:25,AUBERT Patricia,NaN
137,DSI,DIRECTION DES SYSTEMES D'INFORMATION -,NaN,Projet,DSI,Socle,NaN,Normale,2106080011,[DWH] - Monstro - Nouvelles Tables DEST / LDE...,...,Autres,Production,2109080025,08/06/2021 11:25,08/06/2021 12:03,NaN,NaN,10/12/2024 16:48,MESQUITA Pedro,NaN
138,DSI,DIRECTION DES SYSTEMES D'INFORMATION -,NaN,Projet,DSI,Socle,NaN,Normale,2106070034,[DWH] - Monstro - Nouvelles Tables INV / LINV...,...,Autres,Production,2109080024,07/06/2021 18:44,07/06/2021 18:45,NaN,NaN,10/12/2024 16:48,MESQUITA Pedro,NaN


### Merge Databases

In [ ]:
Columns = ['Type', 'Priorité', 'Numéro', 'Titre', 'Statut', 'Domaine', 'Date ouverture', 'Date prise en charge', 'Date résolution',
'Date cloture', 'Groupe Affectation']

In [ ]:
merged_db_asist = pd.concat([db_asist_open[Columns],db_asist[Columns]],ignore_index=True, sort=False)

In [ ]:
merged_db_asist

/usr/local/lib/python3.11/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Type,Priorité,Numéro,Titre,Statut,Domaine,Date ouverture,Date prise en charge,Date résolution,Date cloture,Groupe Affectation
0,Projet,Normale,2503070046,BORDEREAU MEP DEROGATOIRE 11/03/2025,Ouvert,Autres,07/03/2025 17:38,NaN,NaN,NaN,"DSI PROJET, TMA"
1,Projet,Normale,2503120018,[RDEX][1][1SPECEDI] EDI DPV : SUIVI DPV ORDINI...,Pris en charge,EDI_SPEC,12/03/2025 13:24,12/03/2025 13:32,NaN,NaN,"DSI PROJET, TMA"
2,Projet,Normale,2503110035,[DEX] SEPIA - SAG flux stockStatus,Pris en charge,API_SPEC,11/03/2025 17:51,11/03/2025 18:02,NaN,NaN,"DSI PROJET, TMA_DEX"
3,Projet,Normale,2503110013,Check adapter charset to database,Pris en charge,API_STD,11/03/2025 11:30,11/03/2025 13:01,NaN,NaN,"DSI PROJET, TMA_DEX"
4,Projet,Normale,2503100034,[RAUG2501]_Correctif BL spécifiques ALMS pour ...,Pris en charge,Augusta,10/03/2025 16:37,10/03/2025 16:43,NaN,NaN,"DSI PROJET, TMA"
...,...,...,...,...,...,...,...,...,...,...,...
11086,Demande,Haute,2102040024,CODE BARRE etiquette LM2S - BT,Résolu,DT,2021-02-04 00:00:00,2024-06-21 00:00:00,2024-06-21 00:00:00,2024-06-26 00:00:00,"DSI, DSI PROJET"
11087,Projet,Normale,2101220021,Projet global API/BUS : Lot 3 = mise en place ...,Résolu,AUTRES,2021-01-22 00:00:00,2024-12-13 00:00:00,2024-12-13 00:00:00,2024-12-18 00:00:00,DSI PROJET
11088,Incident,Bas,2007060022,"Problem with ""German Umlaute"" in Augusta",Résolu,AUGUSTA,2020-07-06 00:00:00,2020-07-09 00:00:00,2024-03-27 00:00:00,2024-03-30 00:00:00,DSI
11089,Projet,Normale,2003180020,[RAUG] PRO_MARCHANDS : PICKING SUR STOCK AVANCE,Résolu,AUGUSTA,2020-03-18 00:00:00,2020-03-18 00:00:00,2024-08-07 00:00:00,2024-08-09 00:00:00,"DSI PROJET, TMA"


In [ ]:
merged_db_asist.isna().sum()

,0
Type,0
Priorité,0
Numéro,0
Titre,0
Statut,0
Domaine,107
Date ouverture,0
Date prise en charge,21
Date résolution,138
Date cloture,140


In [ ]:
merged_db_asist['Domaine'] = merged_db_asist['Domaine'].fillna('Autres')


#Adding the estimative and Time Spent to Asist Database

In [ ]:
merged_db_asist['Numéro'] = merged_db_asist['Numéro'].astype(str)

# Agrupar os dados por 'Nº Ticket Asist' e somar as colunas relevantes
df_Projects_grouped = df_Projects.groupby('Nº Ticket Asist', as_index=False)[
    ['Σ Original Estimate', 'Σ Time Spent']
].sum()

# Realizar o merge entre as tabelas para adicionar as colunas 'Σ Remaining Estimate' e 'Σ Time Spent'
merged_db_asist = pd.merge(
    merged_db_asist,
    df_Projects_grouped[['Nº Ticket Asist', 'Σ Original Estimate', 'Σ Time Spent']],
    left_on='Numéro',
    right_on='Nº Ticket Asist',
    how='left'
)

# Substituir valores NaN por 0 nas colunas 'Σ Original Estimate', 'Σ Remaining Estimate' e 'Σ Time Spent'
merged_db_asist[['Σ Original Estimate', 'Σ Time Spent']] = merged_db_asist[['Σ Original Estimate', 'Σ Time Spent']].fillna(0)


In [ ]:
merged_db_asist['Projeto_TMA'] = merged_db_asist.apply(
    lambda row: 'TMA' if row['Σ Original Estimate'] < 5 else 'Projet', axis=1
)

In [ ]:
merged_db_asist = merged_db_asist.drop_duplicates(subset=['Numéro'])
merged_db_asist= merged_db_asist[merged_db_asist['Groupe Affectation'].str.contains('TMA', na=False)]
merged_db_asist.reset_index(drop=True, inplace=True)

In [ ]:
merged_db_asist

/usr/local/lib/python3.11/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Type,Priorité,Numéro,Titre,Statut,Domaine,Date ouverture,Date prise en charge,Date résolution,Date cloture,Groupe Affectation,Nº Ticket Asist,Σ Original Estimate,Σ Time Spent,Projeto_TMA
0,Projet,Normale,2503070046,BORDEREAU MEP DEROGATOIRE 11/03/2025,Ouvert,Autres,07/03/2025 17:38,NaN,NaN,NaN,"DSI PROJET, TMA",NaN,0.0,0.00,TMA
1,Projet,Normale,2503120018,[RDEX][1][1SPECEDI] EDI DPV : SUIVI DPV ORDINI...,Pris en charge,EDI_SPEC,12/03/2025 13:24,12/03/2025 13:32,NaN,NaN,"DSI PROJET, TMA",NaN,0.0,0.00,TMA
2,Projet,Normale,2503110035,[DEX] SEPIA - SAG flux stockStatus,Pris en charge,API_SPEC,11/03/2025 17:51,11/03/2025 18:02,NaN,NaN,"DSI PROJET, TMA_DEX",NaN,0.0,0.00,TMA
3,Projet,Normale,2503110013,Check adapter charset to database,Pris en charge,API_STD,11/03/2025 11:30,11/03/2025 13:01,NaN,NaN,"DSI PROJET, TMA_DEX",NaN,0.0,0.00,TMA
4,Projet,Normale,2503100034,[RAUG2501]_Correctif BL spécifiques ALMS pour ...,Pris en charge,Augusta,10/03/2025 16:37,10/03/2025 16:43,NaN,NaN,"DSI PROJET, TMA",2503100034,2.4,0.56,TMA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2856,Projet,Haute,2111230002,[RAUG2402]_RECEPTION : Adaptation sur unité d...,Résolu,AUGUSTA,2021-11-23 00:00:00,2021-11-23 00:00:00,2024-11-14 00:00:00,2024-11-19 00:00:00,"DSI PROJET, TMA",NaN,0.0,0.00,TMA
2857,Projet,Normale,2111040045,[RAUG2401]_Correction taille champ référence e...,Résolu,AUGUSTA,2021-11-04 00:00:00,2021-11-04 00:00:00,2024-05-16 00:00:00,2024-05-21 00:00:00,"DSI PROJET, TMA",NaN,0.0,0.00,TMA
2858,Projet,Bas,2109130016,[RAUG2201] PRODUITS MARCHANDS : PRELEVEMENT M...,Résolu,AUGUSTA,2021-09-13 00:00:00,2021-09-13 00:00:00,2024-03-03 00:00:00,2024-03-06 00:00:00,"DSI PROJET, TMA",NaN,0.0,0.00,TMA
2859,Projet,Bas,2108060018,MCO_Evolution shell intégration EDI propriétai...,Résolu,EDI (Ne pas utiliser),2021-08-06 00:00:00,2021-08-06 00:00:00,2024-10-10 00:00:00,2024-10-14 00:00:00,"DSI PROJET, TMA",2108060018,0.0,0.10,TMA


#Adding TMA/PROJECT to worklogs

In [ ]:
merged_worklogs = worklogs


In [ ]:
worklogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1334 entries, 0 to 1333
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Worklog Id          1334 non-null   int64         
 1   Issue Key           1334 non-null   object        
 2   Issue Summary       1334 non-null   object        
 3   Issue Status        1334 non-null   object        
 4   Start Date          1334 non-null   datetime64[ns]
 5   Update Date         1334 non-null   object        
 6   Author              1334 non-null   object        
 7   Project Name        1334 non-null   object        
 8   Issue Type          1334 non-null   object        
 9   Due Date            159 non-null    object        
 10  Parent Key          1334 non-null   object        
 11  Time Spent (Jours)  1334 non-null   float64       
 12  Week Number         1334 non-null   UInt32        
 13  Year                1334 non-null   UInt32      

In [ ]:
print(merged_db_asist['Numéro'].duplicated().sum())
merged_db_asist = merged_db_asist.drop_duplicates(subset=['Numéro'])


0


In [ ]:
merged_worklogs['Nº Ticket Asist'] = merged_worklogs['Nº Ticket Asist'].astype(str)
merged_db_asist['Numéro'] = merged_db_asist['Numéro'].astype(str)

merged_worklogs = pd.merge(
    merged_worklogs,
    merged_db_asist[['Projeto_TMA','Numéro', 'Domaine', 'Type', 'Σ Original Estimate']],
    left_on='Nº Ticket Asist',
    right_on='Numéro',
    how='left'
)

merged_worklogs.fillna({'Projeto_TMA': 'Unknown', 'Domaine': 'Autres', 'Type': 'Undefined'}, inplace=True)


In [ ]:
#merged_worklogs = pd.concat([worklogs_edited, merged_worklogs])

In [ ]:
merged_worklogs = merged_worklogs.drop_duplicates(subset=['Worklog Id'], keep='last')


In [ ]:
merged_worklogs

,Worklog Id,Issue Key,Issue Summary,Issue Status,Start Date,Update Date,Author,Project Name,Issue Type,Due Date,...,Time Spent (Jours),Week Number,Year,Project,Nº Ticket Asist,Projeto_TMA,Numéro,Domaine,Type,Σ Original Estimate
0,51500,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),2025-01-27 11:07:00,2025-01-27T11:37:05+01:00,Diogo Afonso,STACI : TMA_DEX ( API ),Epic,2025-01-23,...,0.06,5,2025,Augusta,2501150024,Projet,2501150024,API_SPEC,Projet,6.7
1,51514,SA-393,2501240026 - [API] intégrateur API client: sta...,Livré (MEP),2025-01-27 17:07:30,2025-01-27T18:07:35+01:00,Sérgio Matos,STACI : TMA_DEX ( API ),Story,NaN,...,0.12,5,2025,Augusta,2501240026,TMA,2501240026,AUGUSTA,Projet,0.4
2,51534,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),2025-01-28 05:24:15,2025-01-28T13:24:31+01:00,João Silvestre,STACI : TMA_DEX ( API ),Epic,2025-01-23,...,1.00,5,2025,API,2501150024,Projet,2501150024,API_SPEC,Projet,6.7
3,51535,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),2025-01-28 10:24:35,2025-01-28T13:24:39+01:00,João Silvestre,STACI : TMA_DEX ( API ),Epic,2025-01-23,...,0.38,5,2025,API,2501150024,Projet,2501150024,API_SPEC,Projet,6.7
4,51561,SA-342,"2409270009 - Extract-Resubmit easily the ""payl...",QUALIFICATION,2025-01-29 10:26:25,2025-01-29T18:26:32+01:00,João Silvestre,STACI : TMA_DEX ( API ),Epic,NaN,...,1.00,5,2025,API,2409270009,Projet,2409270009,API_STD,Projet,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,53169,RAUG2501-5,2402190004 - [RAUG] Correctif stockage mode ra...,RECETTE HN OK,2025-03-11 14:11:07,2025-03-11T16:11:12+01:00,Diogo Afonso,STACI : RAUG2501,Epic,NaN,...,0.25,11,2025,Augusta,2402190004,TMA,2402190004,Augusta Radio,Projet,4.0
1330,53186,RAUG2501-26,TEST - 2503030040,To Do,2025-03-11 15:48:39,2025-03-11T19:00:09+01:00,Diogo Afonso,STACI : RAUG2501,Tâche,NaN,...,0.38,11,2025,Augusta,2503030040,TMA,2503030040,Augusta,Projet,1.6
1331,53185,RAUG2501-32,Chiffrage - 2503030051,To Do,2025-03-11 18:18:28,2025-03-11T18:53:32+01:00,José Rodrigues,STACI : RAUG2501,Tâche,NaN,...,0.07,11,2025,Augusta,2503030051,TMA,2503030051,Augusta,Projet,0.0
1332,53179,RAUG2501-7,TEST - 2402190004,To Do,2025-03-11 18:21:33,2025-03-11T18:36:46+01:00,Beatriz Tomaz,STACI : RAUG2501,Tâche,NaN,...,0.03,11,2025,Augusta,2402190004,TMA,2402190004,Augusta Radio,Projet,4.0


In [ ]:
#Adding the classification for each dev/tester
Testers = {"Bruno Pires Amado", "Joao Gomes", "Ana Sousa Neto", "Chevalier", "Diogo Afonso"}
Devs = {"Alex Mesquita", "Andre Rodrigues Diogo", "Beatriz Tomaz", "Carlos Freixo", "Daniel Marin",
           "Diogo Albino", "João Monteiro", "João Silvestre", "Joel Alvaredo",
        "José Rodrigues", "Paulo Nakata", "Pedro Barbosa", "rkessal", "Sérgio Matos", "Soraia Esteves"}

all = Testers.union(Devs)

merged_worklogs['Dev/Tester'] = merged_worklogs['Author'].apply(lambda x: 'Tester' if x in Testers else 'Dev')
merged_worklogs['HN/STACI'] = merged_worklogs['Author'].apply(lambda x: 'HN' if x in all else 'STACI')

In [ ]:
merged_worklogs

,Worklog Id,Issue Key,Issue Summary,Issue Status,Start Date,Update Date,Author,Project Name,Issue Type,Due Date,...,Year,Project,Nº Ticket Asist,Projeto_TMA,Numéro,Domaine,Type,Σ Original Estimate,Dev/Tester,HN/STACI
0,51500,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),2025-01-27 11:07:00,2025-01-27T11:37:05+01:00,Diogo Afonso,STACI : TMA_DEX ( API ),Epic,2025-01-23,...,2025,Augusta,2501150024,Projet,2501150024,API_SPEC,Projet,6.7,Tester,HN
1,51514,SA-393,2501240026 - [API] intégrateur API client: sta...,Livré (MEP),2025-01-27 17:07:30,2025-01-27T18:07:35+01:00,Sérgio Matos,STACI : TMA_DEX ( API ),Story,NaN,...,2025,Augusta,2501240026,TMA,2501240026,AUGUSTA,Projet,0.4,Dev,HN
2,51534,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),2025-01-28 05:24:15,2025-01-28T13:24:31+01:00,João Silvestre,STACI : TMA_DEX ( API ),Epic,2025-01-23,...,2025,API,2501150024,Projet,2501150024,API_SPEC,Projet,6.7,Dev,HN
3,51535,SA-389,2501150024 - [DEX] EPOCH D'ENFANCE - SAG flux ...,Livré (MEP),2025-01-28 10:24:35,2025-01-28T13:24:39+01:00,João Silvestre,STACI : TMA_DEX ( API ),Epic,2025-01-23,...,2025,API,2501150024,Projet,2501150024,API_SPEC,Projet,6.7,Dev,HN
4,51561,SA-342,"2409270009 - Extract-Resubmit easily the ""payl...",QUALIFICATION,2025-01-29 10:26:25,2025-01-29T18:26:32+01:00,João Silvestre,STACI : TMA_DEX ( API ),Epic,NaN,...,2025,API,2409270009,Projet,2409270009,API_STD,Projet,27.0,Dev,HN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,53169,RAUG2501-5,2402190004 - [RAUG] Correctif stockage mode ra...,RECETTE HN OK,2025-03-11 14:11:07,2025-03-11T16:11:12+01:00,Diogo Afonso,STACI : RAUG2501,Epic,NaN,...,2025,Augusta,2402190004,TMA,2402190004,Augusta Radio,Projet,4.0,Tester,HN
1330,53186,RAUG2501-26,TEST - 2503030040,To Do,2025-03-11 15:48:39,2025-03-11T19:00:09+01:00,Diogo Afonso,STACI : RAUG2501,Tâche,NaN,...,2025,Augusta,2503030040,TMA,2503030040,Augusta,Projet,1.6,Tester,HN
1331,53185,RAUG2501-32,Chiffrage - 2503030051,To Do,2025-03-11 18:18:28,2025-03-11T18:53:32+01:00,José Rodrigues,STACI : RAUG2501,Tâche,NaN,...,2025,Augusta,2503030051,TMA,2503030051,Augusta,Projet,0.0,Dev,HN
1332,53179,RAUG2501-7,TEST - 2402190004,To Do,2025-03-11 18:21:33,2025-03-11T18:36:46+01:00,Beatriz Tomaz,STACI : RAUG2501,Tâche,NaN,...,2025,Augusta,2402190004,TMA,2402190004,Augusta Radio,Projet,4.0,Dev,HN


# Extracting Insights for Report

In [ ]:
Start_Date = '2025-01-01'
End_Date = '2025-03-12'

In [ ]:
merged_worklogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1334 entries, 0 to 1333
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Worklog Id           1334 non-null   int64         
 1   Issue Key            1334 non-null   object        
 2   Issue Summary        1334 non-null   object        
 3   Issue Status         1334 non-null   object        
 4   Start Date           1334 non-null   datetime64[ns]
 5   Update Date          1334 non-null   object        
 6   Author               1334 non-null   object        
 7   Project Name         1334 non-null   object        
 8   Issue Type           1334 non-null   object        
 9   Due Date             159 non-null    object        
 10  Parent Key           1334 non-null   object        
 11  Time Spent (Jours)   1334 non-null   float64       
 12  Week Number          1334 non-null   UInt32        
 13  Year                 1334 non-nul

In [ ]:
merged_worklogs['Start Date'] = pd.to_datetime(merged_worklogs['Start Date'], errors='coerce')

merged_worklogs_filtered = merged_worklogs[
    (merged_worklogs['Start Date'] >= Start_Date) &
    (merged_worklogs['Start Date'] <= End_Date)
]

### Pivot Table: Time Spent during the month per Type (Project or TMA) - Excluding ECATS

In [ ]:
merged_worklogs_filtered = merged_worklogs_filtered[merged_worklogs_filtered['Project'] != 'ECATS']

In [ ]:
# Create pivot table
pivot_table = pd.pivot_table(
    merged_worklogs_filtered,
    index='Projeto_TMA',
    values='Time Spent (Jours)',
    columns='Domaine',
    aggfunc='sum',
    fill_value=0  # Ensures no NaN values
)

# Convert values to numeric (in case of unexpected types)
pivot_table = pivot_table.apply(pd.to_numeric, errors='coerce')

# Add total column (sum of all values across columns for each row)
pivot_table["Total"] = pivot_table.sum(axis=1)

# Add total row (sum of all values across rows for each column)
pivot_table.loc["Total"] = pivot_table.sum(axis=0)

In [ ]:
pivot_table

Domaine,API_SPEC,API_STD,AUGUSTA,AUGUSTA > EXPEDITION,AUGUSTA > STATISTIQUE,AUTRES,Augusta,Augusta Radio,Autres,E-CATS > WebCat3,EDI (Ne pas utiliser),EDI_SPEC,EDI_STD,New Ecats,Paug,Total
Projeto_TMA,,,,,,,,,,,,,,,,
Projet,10.33,20.11,1.01,0.00,0.10,0.00,11.59,0.00,7.74,0.00,0.0,2.92,0.00,1.23,15.52,70.55
TMA,3.38,4.04,7.02,0.37,1.21,0.09,11.28,0.99,0.31,0.13,0.5,3.04,1.77,0.00,0.00,34.13
Unknown,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,89.41,0.00,0.0,0.00,0.00,0.00,0.00,89.41
Total,13.71,24.15,8.03,0.37,1.31,0.09,22.87,0.99,97.46,0.13,0.5,5.96,1.77,1.23,15.52,194.09


### Time Spent on 'Non Facturable' actions

In [ ]:

df_nonfact = merged_worklogs_filtered[merged_worklogs_filtered['Project Name'] == 'STACI : Gestion de Équipe']

df_grouped = df_nonfact.groupby(['Project']).agg({
    'Time Spent (Jours)': 'sum'
}).reset_index()

df_grouped.loc['Total'] = df_grouped.sum(axis=0)

df_grouped



,Project,Time Spent (Jours)
0,API,6.00
1,Augusta,36.51
2,Todos,8.49
Total,APIAugustaTodos,51.00


### Time Spent per Dev/Tester per week

In [ ]:
Time_per_week = merged_worklogs_filtered.groupby(['Author', 'Project', 'Year', 'Week Number']).agg({'Time Spent (Jours)': 'sum'})

Time_per_week['Status'] = Time_per_week['Time Spent (Jours)'].apply(lambda x: 'OK' if x == 5 else 'NOK')

Time_per_week

Time Spent (Jours) Status
Author        Project Year Week Number                           
Ana Domingues Todos   2025 7                          0.02    NOK
                           8                          0.25    NOK
                           9                          5.14    NOK
                           10                         4.98    NOK
                           11                         0.24    NOK
...                                                    ...    ...
Sérgio Matos  Augusta 2025 7                          1.96    NOK
                           8                          3.86    NOK
                           9                          4.93    NOK
                           10                         4.97    NOK
                           11                         1.97    NOK

[69 rows x 2 columns]

## ECATS

Show per project for ECATS: Time Spent on Development, Original estimate of project, Time of development spent on Bugs opened by Staci, time of development spent on bugs opened by HN, Time spent on Testes vs original estimate, Number of bugs opened by HN and Staci, AVG solution time

 ### Time Spent on Initial development of project (Excluding Bugs)

In [ ]:
dev_filtered = df_Projects[
    ((df_Projects['Issue Type'] != 'Bug') |
    (~df_Projects['Summary'].str.contains('BUG', case=False, na=False))) &
    (df_Projects['Project name'].str.contains('RECATS', case=False, na=False))
]
Dev_time = dev_filtered.groupby(['Project name']).agg({'Σ Time Spent': 'sum', 'Σ Original Estimate': 'sum'})
Dev_time.loc['Total'] = Dev_time.sum(axis=0)
Dev_time

,Σ Time Spent,Σ Original Estimate
Project name,,
STACI : RECATS2374.2,1.88,1.00
STACI : RECATS2375,752.14,446.12
STACI : RECATS2376,545.31,264.30
STACI : RECATS2377,87.69,93.50
STACI : RECATS2378,10.44,316.30
STACI : RECATS2379,27.65,38.00
Total,1425.11,1159.22


### Time Spent on Solving Bugs (BUG + Test)




In [ ]:
bugs_filtered = df_Projects[
    ((df_Projects['Issue Type'] == 'Bug') |
    (df_Projects['Summary'].str.contains('BUG', case=False, na=False))) &
    (df_Projects['Project name'].str.contains('RECATS', case=False, na=False))
]
bug_time = bugs_filtered.groupby(['Project name']).agg({'Σ Time Spent': 'sum'})
bug_time.loc['Total'] = bug_time.sum(axis=0)
bug_time

,Σ Time Spent
Project name,
STACI : RECATS2374.2,2.88
STACI : RECATS2375,108.52
STACI : RECATS2376,110.59
STACI : RECATS2377,2.50
Total,224.49


### Time Spent on Writing and executing Tests


In [ ]:
Tests_filtered = df_Projects[
    ((df_Projects['Issue Type'] == 'Recette/Testing') |
    (df_Projects['Summary'].str.contains('Testing', case=False, na=False))) &
    (df_Projects['Project name'].str.contains('RECATS', case=False, na=False))
]
Tests = Tests_filtered.groupby(['Project name']).agg({'Σ Time Spent': 'sum',  'Σ Original Estimate': 'sum'})
Tests.loc['Total'] = Tests.sum(axis=0)
Tests

,Σ Time Spent,Σ Original Estimate
Project name,,
STACI : RECATS2375,12.88,7.22
STACI : RECATS2376,72.41,17.70
Total,85.29,24.92


### Number of Bugs Opened By Client vs HN


In [ ]:
df_Projects.loc[:, 'HN/STACI'] = df_Projects['Creator'].apply(lambda x: 'HN' if x in all else 'STACI')

<ipython-input-71-0c01f1f762c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Projects.loc[:, 'HN/STACI'] = df_Projects['Creator'].apply(lambda x: 'HN' if x in all else 'STACI')


In [ ]:
Bugs = df_Projects[
    ((df_Projects['Issue Type'] != 'Bug') |
    (~df_Projects['Summary'].str.contains('BUG', case=False, na=False))) &
    (df_Projects['Project name'].str.contains('RECATS', case=False, na=False))
]
Bugs_count = Bugs.groupby(['Project name', 'HN/STACI']).agg({'Σ Time Spent': 'count'}).reset_index()
Bugs_count_unstacked = Bugs_count.set_index(['Project name', 'HN/STACI'])['Σ Time Spent'].unstack(fill_value=0).reset_index()
Bugs_count.loc['Total'] = Bugs_count.sum(axis=0)
Bugs_count

,Project name,HN/STACI,Σ Time Spent
0,STACI : RECATS2374.2,HN,3
1,STACI : RECATS2375,HN,138
2,STACI : RECATS2375,STACI,26
3,STACI : RECATS2376,HN,82
4,STACI : RECATS2376,STACI,38
5,STACI : RECATS2377,HN,20
6,STACI : RECATS2378,HN,42
7,STACI : RECATS2378,STACI,5
8,STACI : RECATS2379,HN,15
Total,STACI : RECATS2374.2STACI : RECATS2375STACI : ...,HNHNSTACIHNSTACIHNHNSTACIHN,369


### AVG Solution Time

In [ ]:
Solution_time = df_Projects[
    ((df_Projects['Issue Type'] != 'Bug') |
    (~df_Projects['Summary'].str.contains('BUG', case=False, na=False))) &
    (df_Projects['Project name'].str.contains('RECATS', case=False, na=False))
]
Solution_time = Solution_time.groupby(['Project name', 'HN/STACI']).agg({'Σ Time Spent': 'mean'}).reset_index().round(2)
Solution_time = Solution_time.set_index(['Project name', 'HN/STACI'])['Σ Time Spent'].unstack(fill_value=0).reset_index()
Solution_time.loc['Total'] = Solution_time.sum(axis=0)
Solution_time

HN/STACI,Project name,HN,STACI
0,STACI : RECATS2374.2,0.63,0.00
1,STACI : RECATS2375,4.71,3.95
2,STACI : RECATS2376,2.65,8.64
3,STACI : RECATS2377,4.38,0.00
4,STACI : RECATS2378,0.10,1.24
5,STACI : RECATS2379,1.84,0.00
Total,STACI : RECATS2374.2STACI : RECATS2375STACI : ...,14.31,13.83


##Machine Learning Methods

Will separate the dataset 'db_asist' in two. The tickets classified as closed will be used during training phase

### Forecast of Time Spent on ticket facturation for project billing optimization - with Random Forest Regressor

In [ ]:
df = merged_db_asist[merged_db_asist['Statut'] == 'Résolu']

In [ ]:
df = df[df['Σ Time Spent'] != 0]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140 entries, 143 to 2859
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Type                  140 non-null    object 
 1   Priorité              140 non-null    object 
 2   Numéro                140 non-null    object 
 3   Titre                 140 non-null    object 
 4   Statut                140 non-null    object 
 5   Domaine               140 non-null    object 
 6   Date ouverture        140 non-null    object 
 7   Date prise en charge  140 non-null    object 
 8   Date résolution       140 non-null    object 
 9   Date cloture          140 non-null    object 
 10  Groupe Affectation    140 non-null    object 
 11  Nº Ticket Asist       140 non-null    object 
 12  Σ Original Estimate   140 non-null    float64
 13  Σ Time Spent          140 non-null    float64
 14  Projeto_TMA           140 non-null    object 
dtypes: float64(2), object(13)

In [ ]:
df['Priorité'].unique()

array(['Normale', 'Haute', 'Bas'], dtype=object)

In [ ]:
df.isna().sum()

,0
Type,0
Priorité,0
Numéro,0
Titre,0
Statut,0
Domaine,0
Date ouverture,0
Date prise en charge,0
Date résolution,0
Date cloture,0


In [ ]:
df.loc[:,'Titre'] = df['Titre'].astype(str)

In [ ]:
# Pré-processamento de texto (Descrição dos tickets)
tfidf = TfidfVectorizer(stop_words='english')
X_text = tfidf.fit_transform(df['Titre'])

tfidf_df = pd.DataFrame(X_text.toarray(), columns=tfidf.get_feature_names_out())


In [ ]:
transformer = ColumnTransformer(
    [('vec', TfidfVectorizer(), df['Titre'])],   # column should be a string or int
    remainder='passthrough'
)

In [ ]:
#Creating independent and dependent variables
X = pd.concat([df[['Priorité', 'Type']], pd.DataFrame(X_text.toarray())], axis=1)

y = df['Σ Time Spent']
y = df.rename(columns={"Σ Time Spent": "Time Spent"})["Time Spent"]


In [ ]:
y

,Time Spent
143,0.12
247,1.28
251,0.44
259,3.93
280,0.25
...,...
2598,1.55
2816,0.03
2819,0.38
2832,0.56


In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform="pandas")

le = LabelEncoder()

#X_encoded = enc.fit_transform(df[['Priorité', 'Type']])

le_priorite = LabelEncoder()
df['Priorité_encoded'] = le_priorite.fit_transform(df['Priorité'])

le_type = LabelEncoder()
df['Type_encoded'] = le_type.fit_transform(df['Type'])
X_encoded = df[['Priorité_encoded', 'Type_encoded']]


X = pd.concat([X_encoded, tfidf_df], axis=1)
X


,Priorité_encoded,Type_encoded,0240_,1apispec,1specadi,1specedi,2405150001,2757,3476,3476_urmet,...,webservices,wms,xml,yoplait,écran,édition,éditions,état,évolution,évolutions
143,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.439577,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y

,Time Spent
143,0.12
247,1.28
251,0.44
259,3.93
280,0.25
...,...
2598,1.55
2816,0.03
2819,0.38
2832,0.56


In [ ]:

df = df.dropna(subset=['Priorité'])  # Ajuste 'priority' conforme necessário
X = X.loc[df.index]  # Atualize X para manter as mesmas linhas de df
y = df['Σ Time Spent']  # Atualize y com a coluna 'priority' de df


y = y[X.index]

print(len(X), len(y))

140 140


In [ ]:
#Training the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")


mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 1.0686571428571428
Mean Squared Error (MSE): 3.0700466035714293
R² Score: -0.013847154398396011


The values found for the model sugests that the model is not performing well. One of the hipothesis we don't have enough data to tests and the model would need to be applied when we gather enough data.

Another solution is to analyse which variable has a higher correlation to the time spent so we could gather some insights

In [ ]:
label_encoder = LabelEncoder()

categorical_columns = df.select_dtypes(include=['object']).columns


for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

In [ ]:

corr_matrix = df.corr()

time_spent_corr = corr_matrix["Σ Time Spent"]

time_spent_corr = time_spent_corr.drop("Σ Time Spent").sort_values(ascending=False)

print(time_spent_corr)

Σ Original Estimate     0.825785
Date cloture            0.170782
Groupe Affectation      0.162588
Date résolution         0.162560
Date prise en charge    0.139463
Type_encoded            0.073484
Type                    0.073484
Titre                   0.004734
Date ouverture         -0.009996
Numéro                 -0.016241
Nº Ticket Asist        -0.016241
Domaine                -0.269807
Priorité_encoded       -0.276712
Priorité               -0.276712
Projeto_TMA            -0.677919
Statut                       NaN
Name: Σ Time Spent, dtype: float64


As we can see only original estimate has a strong correlation to our taget variable, as we can see the variables that the management wanted to use (Priorité and Type) have a really low correlation indication that it cannot be used for a prediction of Time Spent

In [ ]:
#Creating independent and dependent variables
X = df['Σ Original Estimate']

y = df['Σ Time Spent']
y = df.rename(columns={"Σ Time Spent": "Time Spent"})["Time Spent"]


In [ ]:
# Se você tiver uma única coluna em X_train, converta para 2D
X = X.values.reshape(-1, 1)

In [ ]:
#Training the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")


mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 1.0686571428571428
Mean Squared Error (MSE): 3.0700466035714293
R² Score: -0.013847154398396011


The values are the same even with a higher correlated variable, we will test now to apply Linear Regression to test how it will perform

In [ ]:
# Dividindo os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()

# Treinando o modelo com os dados de treinamento
model.fit(X_train, y_train)


LinearRegression()

In [ ]:
# Fazendo previsões com os dados de teste
y_pred = model.predict(X_test)

# Calculando as métricas de desempenho
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibindo os resultados
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 0.7943225294956073
Mean Squared Error (MSE): 1.7588233889912683
R² Score: 0.4191690491135399


In [ ]:
# Validando o modelo com validação cruzada (cross-validation)
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')

# Exibindo os resultados da validação cruzada
print(f"Validação Cruzada - MSE médio (5 folds): {-np.mean(cv_scores)}")

Validação Cruzada - MSE médio (5 folds): 1.156420863946607


Linear Regression works better than Random Forest, We can try to apply Hiperparameters Tunning to improve the performance

In [ ]:
param_grid = {
    'fit_intercept': [True, False],
}

In [ ]:
model = LinearRegression()


In [ ]:
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')


In [ ]:
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False]},
             scoring='neg_mean_squared_error')

In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test)


In [ ]:

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 0.7968805096704569
Mean Squared Error (MSE): 1.737450959450604
R² Score: 0.42622704518668597


The performace had a small improvement after tunning the hiperparameters, the model can be used but will have a significant error, for a better prediction was suggested to the client to gatter more data from the projects to be able to have sufficient info for a better model.
